In [3]:
pip install psycopg2


Note: you may need to restart the kernel to use updated packages.


In [1]:
import psycopg2 
from psycopg2.extras import execute_values
import pandas as pd

In [2]:
conn = psycopg2.connect(host = 'database-2.cnvl9hnkloyz.us-east-2.rds.amazonaws.com',user ='postgres',password='shyambaba',database='postgres')
mycursor = conn.cursor()
conn.set_session(autocommit = True)

In [3]:
data = pd.read_json("train.json.zip")
data.head(2)

,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."


In [13]:
#Create table for dishes

query1 ="CREATE TABLE dishes (dish_id INT,cuisine_id INT,PRIMARY KEY (dish_id))"
mycursor.execute(query1)


In [10]:
#Inserting data into dishes

data1= data[['cuisine']]
data1.drop_duplicates(subset='cuisine',inplace=True)
data1 = data1.reset_index(drop = True).reset_index()
datanew = data[['id','cuisine']]#dataframe

#DataFrame for dishes 
table1 = datanew.replace(to_replace=list(data1['cuisine']),value =list(data1['index'])) 
l1= list(zip(table1['id'],table1['cuisine']))#creating list

execute_values(mycursor,"INSERT INTO dishes (dish_id,cuisine_id) VALUES %s",l1)

C:\Users\KIRTY\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [26]:
#Create table for Cuisine
query2 ="CREATE TABLE Cuisine (cuisine_id INT PRIMARY KEY,cuisine_name VARCHAR(40))"
mycursor.execute(query2)

In [11]:
#Insert values into Cuisine
l2= list(zip(data1['index'],data1['cuisine'])) #creating list

execute_values(mycursor, "INSERT INTO Cuisine (cuisine_id,cuisine_name) VALUES %s",l2)

In [28]:
#Create table for Ingredients

query3 ="CREATE TABLE Ingredients (ingred_id INT PRIMARY KEY,ingred_name VARCHAR)"
mycursor.execute(query3)

In [19]:
#Inserting values into INgredients

e= data.explode('ingredients')
e.drop_duplicates(subset='ingredients',inplace=True)
data2 =pd.DataFrame(e['ingredients']).reset_index(drop=True).reset_index()
l3 = list(zip(data2['index'],data2['ingredients'])) #creating list
execute_values(mycursor,"INSERT INTO Ingredients (ingred_id,ingred_name) VALUES %s",l3)

In [31]:
#Create table for Recipe
query3 ="CREATE TABLE Recipe (r_id INT , dish_id INT, ingred_id INT)"
mycursor.execute(query3)

In [14]:
#Inserting values into Recipe
cuisine = list(data1['cuisine'])
cuisine_id =list(data1['index'])
ingredients=list(data2['ingredients'])
ingred_id =list(data2['index'])

table2 =data.explode('ingredients').replace(to_replace=cuisine,value =cuisine_id)#assigning id to cuisine
table3 = table2['ingredients'].replace(to_replace=ingredients,value =ingred_id)#assigning id to ingredients
df=[table2[['id','cuisine']],table3]#creating dataframe
res = pd.concat(df,axis=1)
res.reset_index(inplace=True)



In [15]:
l4 =res[['index','id','ingredients']]
l5=list(zip(*map(l4.get,l4)))#creating list
execute_values(mycursor,"INSERT INTO Recipe (r_id, dish_id, ingred_id) VALUES %s",l5)